In [54]:
sc.stop()

In [1]:
import time
from pprint import pprint

import nltk
stopwords = nltk.corpus.stopwords.words('english')

# Create PySpark context
from pyspark import  SparkContext, SQLContext, Row
sc = SparkContext('local[*]', '--executor-memory=10g pyspark')
sc.defaultParallelism

# stopwords_spark = sc.broadcast(stopwords)

16

In [2]:
book_filename = "BX-Books.csv"
book_filename = "books-mini.csv"
book_filename = "books-small.csv"
book_filename = "books-medium.csv"
book_filename = "books-large.csv"
book_filename = "books-xlarge.csv"
book_filename = "books-xxlarge.csv"
# book_filename = "books-xlarge-tail.csv"
book_filename = "books-xlarge2.csv"

book_raw_data = sc.textFile(book_filename, minPartitions=6)
book_raw_data_header = [ word.replace("\"", "") for word in book_raw_data.take(1)[0].split(';')[1:3] ] # Used to skip the header
book_raw_data =  book_raw_data.map(lambda line: [ word.replace("\"", "") for word in line.split(";")[1:3]]) \
                            .filter(lambda line: line != book_raw_data_header) \
                            .map(lambda line: "%s %s" % (line[0], line[1])) \
                            .map(lambda line: [word for word in line.split(" ") if word.lower() not in stopwords]).zipWithIndex()
book_raw_data.take(2)

[([u'Flight',
   u"Stoneman's",
   u'Son',
   u'(The',
   u'Flight',
   u'Stoneman)',
   u'Terence',
   u'Munsey'],
  0),
 ([u'Son',
   u'Smaller',
   u'Hero',
   u'(New',
   u'Canadian',
   u'Library)',
   u'M.',
   u'Richler'],
  1)]

In [3]:
sqlContext = SQLContext(sc)

book_table = book_raw_data.map(lambda p: Row(idx=p[1], title=p[0]))
schema_books = sqlContext.createDataFrame(book_table)
schema_books.registerTempTable("books")

# example
book = sqlContext.sql("SELECT * FROM books WHERE idx = 400 LIMIT 1")
book.collect()

[Row(idx=400, title=[u'Mouth', u'Mouth', u'Kevin', u'Elyot'])]

In [4]:
titles = sqlContext.sql("SELECT title FROM books").map(lambda row: row[0])
titles.take(1)

[[u'Flight',
  u"Stoneman's",
  u'Son',
  u'(The',
  u'Flight',
  u'Stoneman)',
  u'Terence',
  u'Munsey']]

In [5]:
start = time.time()
from pyspark.mllib.feature import HashingTF
hashingTF = HashingTF()
tf = hashingTF.transform(titles)
from pyspark.mllib.feature import IDF
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)
end = time.time()

In [6]:
# l = map(lambda x: max(x.values), tfidf.take(1000000))
# max(l)

In [7]:
print end - start


2.0401930809


In [8]:
start = time.time()

from pyspark_lsh import lsh
# import pyspark_lsh
# reload(pyspark_lsh)

# p : integer, larger than the largest value in data.
# m : integer, number of bins for hashing.
# n : integer, number of rows to split the signatures into.
# b : integer, number of bands. Each band will have (n / b) element
# c : integer, minimum allowable cluster size.
lsh_model = lsh.run(tfidf, p = 100, m = 100, n = 50, b = 10, c = 5)
end = time.time()
print end - start

bp 1
6
bp 2
6
bp 3
bp 4
6
bp 5
6
bp 6
12
bp 7
12
bp 8
24.3293390274


In [9]:
start = time.time()
buckets_to_check = lsh_model.scores.filter(lambda bucket_score: bucket_score[1] > 200).collect()
# print buckets_to_check
print len(buckets_to_check)
end = time.time()

1869


In [10]:
# 598
# 1337

In [11]:
print end - start

2221.714535


In [12]:
lsh_model.scores.getNumPartitions()

12

In [13]:
# https://www.quora.com/How-do-you-implement-a-Disjoint-set-data-structure-in-Python
global_merge_list = {}

def parent(rep, v):
    if rep[v] == v:
        return v
    rep[v] = parent(rep, rep[v])
    return rep[v]

def merge(rep, L):
    for edge in L:
        u, v = edge
        if u not in rep:
            rep[u] = u
        if v not in rep:
            rep[v] = v
        rep[parent(rep, v)] = parent(rep, u)
    return rep

def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [14]:
start = time.time()
for (bucket_idx, score) in buckets_to_check:
    bstart = time.time()
    
    bv = lsh_model.buckets_vectors.filter(lambda bv: bv[0] == bucket_idx)
    # print bv.collect()

    values = bv.groupByKey().collect()[0][1]
    
#     books_in_bucket = []
#     for x in bv.collect():
#         books_in_bucket.append(sqlContext.sql("select * from books where idx = %s LIMIT 1" % x[1]) \
#                                         .map(lambda row: (row[0], row[1])).collect()[0])
    books_in_bucket = sqlContext.sql("select * from books where idx in %s" % str(values.data).replace('[', '(').replace('L', '').replace(']', ')')) \
                                  .map(lambda row: (row[0], row[1])).collect()
    
    to_merge = []
    for idx1, book1 in enumerate(books_in_bucket):
        (gidx1, title1) = book1
        for idx2 in range(idx1+1, len(books_in_bucket)):
            book2 = books_in_bucket[idx2]
            (gidx2, title2) = book2

            x = set(title1)
            y = set(title2)

            if jaccard(x,y) > 0.85:
                to_merge.append((gidx1, gidx2))
    
    #pprint(to_merge)
    
    merge(global_merge_list, to_merge)
    
    print(time.time() - bstart)

#pprint(global_merge_list)

end = time.time()
print end - start

0.972131967545
0.630781888962
0.802431106567
0.79522895813
0.765422105789
0.733091115952
0.5484790802
0.765599012375
0.728838920593
0.727472066879
0.728400945663
0.529352903366
0.762191057205
0.7112429142
0.763869047165
0.72635102272
0.728669166565
0.528126001358
0.52950501442
0.709020137787
0.717177152634
0.530468940735
0.709718942642
0.531512975693
0.700841903687
0.708933115005
0.716392993927
0.713235855103
0.512382030487
0.732991933823
0.720467090607
0.723659038544
0.661617040634
0.724543094635
0.726704120636
0.715128898621
0.707808017731
0.712588071823
0.6969871521
0.706522941589
0.685214996338
0.710812091827
0.516491889954
0.713132858276
0.702546834946
0.696578979492
0.784993886948
0.718825101852
0.744344949722
0.717131137848
0.699106931686
0.689143896103
0.683176040649
0.709452867508
0.573600053787
0.625687122345
0.505009889603
0.723686933517
0.500033855438
0.706146001816
0.700882911682
0.689821004868
0.512333869934
0.678271055222
0.692149162292
0.684293031693
0.711277008057
0.69

In [15]:
count = 0
for merge1, merge2 in global_merge_list.iteritems():
    if merge1 != merge2:
        book = sqlContext.sql("SELECT * FROM books WHERE idx = %s LIMIT 1" % merge1)
        print book.collect()
        book = sqlContext.sql("SELECT * FROM books WHERE idx = %s LIMIT 1" % merge2)
        print book.collect()
        print
        count += 1
        if count > 10: break

[Row(idx=8192, title=[u"Fortune's", u'Wheel', u'Cynthia', u'Voigt'])]
[Row(idx=3176, title=[u"Fortune's", u'Wheel', u'Cynthia', u'Voigt'])]

[Row(idx=24578, title=[u'Silicon', u'Snake', u'Oil:', u'Second', u'Thoughts', u'Information', u'Highway', u'Clifford', u'Stoll'])]
[Row(idx=17567, title=[u'Silicon', u'Snake', u'Oil:', u'Second', u'Thoughts', u'Information', u'Highway', u'Clifford', u'Stoll'])]

[Row(idx=8199, title=[u'Secret', u'Garden', u'(Penguin', u'Classics)', u'Frances', u'Hodgson', u'Burnett'])]
[Row(idx=1145, title=[u'Secret', u'Garden', u'(Penguin', u'Popular', u'Classics)', u'Frances', u'Hodgson', u'Burnett'])]

[Row(idx=47788, title=[u'Stones', u'Summer', u'Dow', u'Mossman'])]
[Row(idx=4610, title=[u'Stones', u'Summer', u'Dow', u'Mossman'])]

[Row(idx=24612, title=[u'Tan', u'Veloz', u'Como', u'El', u'Deseo', u'Laura', u'Esquivel'])]
[Row(idx=18724, title=[u'Tan', u'Veloz', u'Como', u'El', u'Deseo', u'Laura', u'Esquivel'])]

[Row(idx=24619, title=[u'Unholy', u'Fire', u'W

In [16]:
import pickle
f=open('merge_dict_v1_p100m100n50b10c5_xlarge2', 'wb')
pickle.dump(global_merge_list, f, protocol=2)
f.close()

----

----

In [ ]:
len(global_merge_list)

In [ ]:
bv = lsh_model.buckets_vectors.filter(lambda bv: bv[0] == 1585)
values = bv.groupByKey().collect()[0][1]
values

In [ ]:
sqlContext.sql("select * from books where idx in %s" % str(values.data).replace('[', '(').replace('L', '').replace(']', ')')) \
          .map(lambda row: (row[0], row[1])).collect()

In [ ]:
nltk.download()